In [1]:
import warnings
warnings.filterwarnings("ignore", message="Gym has been unmaintained since")

import sys
sys.path.append("../env")  # to read hospital_env.py
from hospital_env import HospitalEnv

import gymnasium as gym
import numpy as np
import torch

from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv

# ------------------------------
# Seting seeds for reproducibility
# ------------------------------
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

# --- Create wrapped environment ---
def make_env():
    env = HospitalEnv()
    env.reset(seed=SEED)   # seed the environment here
    env = Monitor(env)      # important for SB3 logging
    return env

env = DummyVecEnv([make_env])

# --- Creating DQN agent ---
model = DQN(
    "MlpPolicy",       
    env,
    learning_rate=5e-4,
    gamma=0.95,
    batch_size=64,
    buffer_size=50000,
    exploration_initial_eps=1.0,
    exploration_final_eps=0.1,
    exploration_fraction=0.1,   # epsilon decay
    target_update_interval=1000,
    verbose=1,
    seed=SEED                  # seed SB3 agent
)

# --- Training agent ---
model.learn(total_timesteps=50000)

# --- Saving trained model ---
model.save("../models/dqn_hospital_sb3")
print("Model saved to models/dqn_hospital_sb3.zip")

# --- Evaluation ---
eval_env = DummyVecEnv([make_env])
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward:.2f} ± {std_reward:.2f}")

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


Using cpu device
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 30       |
|    ep_rew_mean      | 1.64e+03 |
|    exploration_rate | 0.978    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 1533     |
|    time_elapsed     | 0        |
|    total_timesteps  | 120      |
| train/              |          |
|    learning_rate    | 0.0005   |
|    loss             | 58.2     |
|    n_updates        | 4        |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 30       |
|    ep_rew_mean      | 1.65e+03 |
|    exploration_rate | 0.957    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 758      |
|    time_elapsed     | 0        |
|    total_timesteps  | 240      |
| train/              |          |
|    learning_rate    | 0.0005   |
|    loss             | 62.1     |
|  

In [2]:
import sys
import numpy as np
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor

# Adding environment path
sys.path.append("../env")  
from hospital_env import HospitalEnv

# -------------------------------
# Creating wrapped evaluation environment
# -------------------------------
def make_env():
    env = HospitalEnv()
    env = Monitor(env)  # SB3 logging wrapper
    return env

eval_env = DummyVecEnv([make_env])

# -------------------------------
# Loading trained DQN model
# -------------------------------
model = DQN.load("../models/dqn_hospital_sb3", env=eval_env)

# -------------------------------
# Evaluation parameters
# -------------------------------
n_episodes = 40  # number of episodes to evaluate
threshold_times = {"red": 15, "yellow": 30}

# Storage for metrics
rewards_per_episode = []
red_waits, yellow_waits = [], []
queue_lengths = {"red": [], "yellow": []}
total_served_count = 0

# -------------------------------
# Model Evaluation
# -------------------------------
for ep in range(n_episodes):
    obs = eval_env.reset()  # only obs returned
    done = False
    episode_reward = 0

    while not done:
        # Predicting action
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, info = eval_env.step(action)
        episode_reward += reward[0]

        # Accessing raw environment
        env = eval_env.envs[0].unwrapped

        # Recording last served wait times
        if env.last_served_wait_times["red"] > 0:
            red_waits.append(env.last_served_wait_times["red"])
            total_served_count += 1
        if env.last_served_wait_times["yellow"] > 0:
            yellow_waits.append(env.last_served_wait_times["yellow"])
            total_served_count += 1

        # Recording queue lengths
        queue_lengths["red"].append(len(env.red_queue))
        queue_lengths["yellow"].append(len(env.yellow_queue))

    rewards_per_episode.append(episode_reward)

# -------------------------------
# Computing metrics
# -------------------------------
avg_reward = np.mean(rewards_per_episode)
avg_wait_red = np.mean(red_waits) if red_waits else 0
avg_wait_yellow = np.mean(yellow_waits) if yellow_waits else 0

pct_red_within = 100 * sum(w <= threshold_times["red"] for w in red_waits) / len(red_waits) if red_waits else 0
pct_yellow_within = 100 * sum(w <= threshold_times["yellow"] for w in yellow_waits) / len(yellow_waits) if yellow_waits else 0

queue_stats = {cat: {"avg": np.mean(qs), "max": np.max(qs)} for cat, qs in queue_lengths.items()}

# -------------------------------
# Printing results
# -------------------------------
print(f"Average reward per episode: {avg_reward:.2f}")
print(f"Average wait times (Red, Yellow): {avg_wait_red:.2f}, {avg_wait_yellow:.2f}")
print(f"Percentage served within thresholds (Red, Yellow): {pct_red_within:.2f}%, {pct_yellow_within:.2f}%")
print("Queue stats (average and max lengths):")
for cat, stats in queue_stats.items():
    print(f"  {cat.capitalize()}: avg={stats['avg']:.2f}, max={stats['max']:.2f}")


Average reward per episode: 1895.70
Average wait times (Red, Yellow): 3.90, 5.86
Percentage served within thresholds (Red, Yellow): 100.00%, 100.00%
Queue stats (average and max lengths):
  Red: avg=1.85, max=9.00
  Yellow: avg=6.54, max=25.00


In [3]:
import warnings
warnings.filterwarnings("ignore", message="Gym has been unmaintained since")
from stable_baselines3 import DQN
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.evaluation import evaluate_policy
from hospital_env import HospitalEnv  # replace with your actual path

# Creating and wrapping environment
env = HospitalEnv()
env = Monitor(env)

# Convergence check function
def check_convergence(mean_rewards, threshold=10):
    """
    Checks if the last two rewards differ by less than threshold.
    Returns True = converged.
    """
    if len(mean_rewards) < 2:
        return False
    return abs(mean_rewards[-1] - mean_rewards[-2]) < threshold

# Loading the trained model
model = DQN.load("../models/dqn_hospital_sb3", env=env)

# Evaluating over timesteps
mean_rewards_history = []
timesteps_history = []

for t in range(0, 200000, 10000):  
    
    mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=10, deterministic=True)
    mean_rewards_history.append(mean_reward)
    timesteps_history.append(t)

    print(f"At {t} timesteps → mean reward: {mean_reward:.2f}")

    if check_convergence(mean_rewards_history):
        print(f"\nModel converged at ~{t} timesteps.\n")
        break


Wrapping the env in a DummyVecEnv.
At 0 timesteps → mean reward: 1940.10
At 10000 timesteps → mean reward: 1869.30
At 20000 timesteps → mean reward: 1874.40

Model converged at ~20000 timesteps.

